# 02 — Modeling & Forecast

Fit SARIMAX, evaluate on a holdout set, and visualize forecast vs. actual.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

DATA_PATH = Path('../data/prepared_sales.csv')
df = pd.read_csv(DATA_PATH, parse_dates=['Date']).set_index('Date')

# Train/test split (last 60 days as test)
train = df.iloc[:-60]
test = df.iloc[-60:]

# Exogenous regressors
exog_cols = ['Promo','Holiday']
exog_train = train[exog_cols]
exog_test = test[exog_cols]

# Simple SARIMAX with weekly seasonality (7)
model = SARIMAX(train['Sales'], order=(1,1,1), seasonal_order=(1,1,1,7), exog=exog_train, enforce_stationarity=False, enforce_invertibility=False)
res = model.fit(disp=False)

# Forecast horizon = len(test)
forecast = res.get_forecast(steps=len(test), exog=exog_test)
fc_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# Metrics
mae = mean_absolute_error(test['Sales'], fc_mean)
rmse = mean_squared_error(test['Sales'], fc_mean, squared=False)
mape = (np.abs((test['Sales'] - fc_mean) / test['Sales']).replace([np.inf, -np.inf], np.nan).dropna()).mean() * 100
print({'MAE': mae, 'RMSE': rmse, 'MAPE_%': mape})

# Plot forecast vs actual
plt.figure()
train['Sales'].plot(label='Train')
test['Sales'].plot(label='Actual')
fc_mean.plot(label='Forecast')
plt.fill_between(conf_int.index, conf_int.iloc[:,0], conf_int.iloc[:,1], alpha=0.2)
plt.title('Forecast vs Actual (Test Window)')
plt.legend()
plt.show()

# Save forecast
out = pd.DataFrame({'Date': test.index, 'Actual': test['Sales'].values, 'Forecast': fc_mean.values})
out.to_csv('../data/forecast_vs_actual.csv', index=False)
print('Saved ../data/forecast_vs_actual.csv')


### Notes
- Tune (p,d,q)(P,D,Q,7) via grid search or AIC if needed.
- Optional: Try Prophet (install `prophet`) and compare RMSE/MAPE.